<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/Output%20Parsers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Output parsers en Langchain**
---
Imagina que le preguntas a un LLM "¿Cuáles son los tres planetas más cercanos al sol?" y te responde: "Mercurio, Venus y la Tierra son los planetas más cercanos al sol". Si bien la respuesta es correcta para un humano, para que tu programa pueda usar esa información, lo ideal sería tenerla en un formato más manejable, como una lista o un objeto JSON. Aquí es donde entran en juego los Output Parsers.  

Los Output Parsers te permiten “forzar” o “guiar” al modelo para que devuelva la información según un formato deseado (por ejemplo, un JSON con campos específicos, una lista, etc).  

LangChain ofrece una variedad de Output Parsers preconstruidos para diferentes necesidades. Algunos de los mas usados son estos:

- **StrOutputParser:** Convierte la salida a string

-   **StructuredOutputParser:** Analiza la salida en una estructura predefinida con campos.
    
-   **OutputFixingParser:** Corrige errores en el parsing estructurado usando un LLM.

- RetryWithError
    
-   **CommaSeparatedListOutputParser:** Convierte la salida en una lista de strings separados por comas
    
-   **EnumOutputParser:** Espera que el LLM elija de un conjunto de opciones.
    
-   **BooleanOutputParser:** Analiza la salida como Verdadero o Falso.
    
-   **PydanticOutputParser:** Convierte la salida a un modelo de datos Pydantic.
    
-   **RegexParser:** Extrae información usando expresiones regulares.

- JSON ??
- YAML
- XML
-DATETIME

tODOS AQUI: https://python.langchain.com/docs/concepts/output_parsers/




OutputFixingParser: ?

with_structured_output  solo para llm que lo soportan (openai ...)





#**0. Preparando el entorno del cuaderno**



In [1]:
%%capture --no-stderr

# Importar la librería `userdata` de Google Colab.
# Esta librería se utiliza para acceder a datos de usuario almacenados de forma segura en el entorno de Colab.
from google.colab import userdata

# Obtener las claves API de diferentes servicios desde el almacenamiento seguro de Colab.
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')

# Instalar las librerías necesarias usando pip.
# El flag `-qU` instala en modo silencioso (`-q`) y actualiza las librerías si ya están instaladas (`-U`).
%pip install langchain -qU  # Instalar la librería principal de LangChain.

# Instalar las integraciones de LangChain con diferentes proveedores de LLMs.
%pip install langchain-openai -qU
%pip install langchain-groq -qU
%pip install langchain-google-genai -qU
%pip install langchain-huggingface -qU

# Importar las clases necesarias de LangChain para crear plantillas de prompt.
# `ChatPromptTemplate` es la clase base para plantillas de chat.
# `SystemMessagePromptTemplate` se usa para mensajes del sistema (instrucciones iniciales).
# `HumanMessagePromptTemplate` se usa para mensajes del usuario.
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate


# # Importamos las clases necesarias para trabajar con cadenas
# from langchain.chains import LLMChain

# Importar las clases para interactuar con los diferentes LLMs a través de LangChain.
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEndpoint

# Importamos la libreria para formatear mejor la salida
from IPython.display import Markdown, display

# **1. StructuredOutputParser**
Un analizador de salida estructurado está diseñado para extraer y estructurar información de texto no estructurado o semiestructurado. Esto es adecuado para situaciones que requieren la extracción de múltiples campos.

Permite analizar la salida del LLM en una estructura predefinida. Le proporcionas un esquema (generalmente una lista de ResponseSchema que definen los campos esperados) y el parser intenta extraer la información y mapearla a ese esquema.

Aunque el analizador Pydantic (explicado a continuación) ofrece capacidades más sólidas, StructuredOutputParser es ventajoso para su uso con modelos más simples.

**Casos de uso:** Extraer información específica de un texto, como atributos de un producto, detalles de un evento, o datos de contacto. Es muy versátil para obtener datos estructurados.

**Ventaja:** Flexible y permite definir la estructura esperada de la salida.

Veamos un ejemplo en el que el resultado será un diccionario estructurado con los campos nombre, edad y email, listo para ser utilizado en la aplicación.

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

#Creamos un esquema de respuesta (ResponseSchema) para cada campo que queremos extraer:
response_schemas = [
    ResponseSchema(name="nombre", description="El nombre del usuario"),
    ResponseSchema(name="edad", description="La edad del usuario"),
    ResponseSchema(name="email", description="El correo electrónico del usuario")
]

# Crear el StructuredOutputParser
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

**Obtener instrucciones de formato (opcional pero recomendado):** Muchos parsers tienen un método get\_format\_instructions() que devuelve texto que puedes incluir en tu prompt para guiar al LLM sobre el formato esperado.



In [ ]:
# Obtener el formato de instrucciones del parser
format_instructions = output_parser.get_format_instructions()
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"nombre": string  // El nombre del usuario\n\t"edad": string  // La edad del usuario\n\t"email": string  // El correo electrónico del usuario\n}\n```'

In [ ]:
# Crear el prompt
prompt = PromptTemplate(
    template="Genera información de un usuario ficticio.\n{format_instructions}\n",
    input_variables=[],
    partial_variables={"format_instructions": format_instructions}
)

### Sobre las `partial variables`
Las **partial\_variables** dentro de un PromptTemplate son una forma de **pre-cargar o fijar ciertos valores dentro de la plantilla del prompt antes de que se proporcionen las variables de entrada principales, es decir cuando se crea el propmpt template y antes de que se formatee**

Piénsalo de esta manera: un PromptTemplate es como una plantilla de texto con "huecos" que necesitas llenar para crear un prompt completo para el LLM. Hay dos formas principales de llenar estos huecos:

-   **input\_variables:** Estas son las variables que **cambian** cada vez que utilizas el prompt. Son los datos específicos que quieres que el LLM procese en cada llamada.
    
-   **partial\_variables:** Estas son las variables que tienen un valor **fijo** o **predefinido** para un uso particular del PromptTemplate. No cambian con cada llamada a la cadena o LLM que usa este prompt.
    

**En el codigo anterior:**



```
prompt = PromptTemplate(
            template="Genera información de un usuario ficticio.\n{format_instructions}\n",
            input_variables=[],
            partial_variables={"format_instructions": format_instructions}
```
En otras palabras, son una forma de incluir en el prompt template una parte fija, pero mediante una variable, lo que proporciona mas flexibilidad programatica.
En este caso metemos en el prompt, las instruciones que nos ha devuelto el parser.

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=1)

respuesta = llm.invoke(prompt.format()) # la llamada no requiere ningun parámetro
respuesta_analizada = output_parser.parse(respuesta.content)
respuesta_analizada



{'nombre': 'Laura Gómez', 'edad': '28', 'email': 'laura.gomez@example.com'}

Vamos a diseñar una consulta para un modelo de lenguaje (LLM) que sirva como base para generar un test de preguntas sobre un tema específico. Proporcionaremos el tema y un nivel de dificultad (bajo, medio, alto), y el LLM deberá generar el texto de la pregunta, tres opciones de respuesta y el índice de la respuesta correcta. La respuesta debe estar estructurada en un diccionario con el siguiente formato:

```
{
    "pregunta": "Texto de la pregunta",
    "opciones": ["Opción 1", "Opción 2", "Opción 3"],
    "respuesta_correcta": índice_de_la_opción_correcta
}
```

Este formato permitirá una fácil interpretación y uso de la pregunta generada.


In [6]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

#Creamos un esquema de respuesta (ResponseSchema) para cada campo que queremos extraer:
response_schemas = [
            ResponseSchema(name="pregunta", description="Texto de la pregunta generada."),
            ResponseSchema(name="opciones", description="Lista de tres opciones de respuesta."),
            ResponseSchema(name="respuesta_correcta", description="Índice de la opción correcta (0, 1 o 2).")
                    ]

# Crear el StructuredOutputParser
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Obtener el formato de instrucciones del parser
format_instructions = output_parser.get_format_instructions()
format_instructions

# Crear el prompt
texto_prompt = """
        Genera una pregunta de test sobre el tema: {tema}.
        El nivel de dificultad debe ser: {nivel}.
        La pregunta debe tener tres opciones de respuesta y una respuesta correcta.

        {format_instructions}
        """

prompt_template = PromptTemplate(
    template=texto_prompt,
    input_variables=["tema", "nivel"],
    partial_variables={"format_instructions": format_instructions}
)

# Generar el prompt para el LLM
prompt = prompt_template.format(tema="LangChain", nivel="medio")
print("Prompt generado:\n", prompt)

# Instanciamos el modelo
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=1)

# Hacemos la llamada
respuesta = llm.invoke(prompt)

# Analizamos la respuesta
respuesta_analizada = output_parser.parse(respuesta.content)
respuesta_analizada

Prompt generado:
 
        Genera una pregunta de test sobre el tema: LangChain.
        El nivel de dificultad debe ser: medio.
        La pregunta debe tener tres opciones de respuesta y una respuesta correcta.
        
        The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"pregunta": string  // Texto de la pregunta generada.
	"opciones": string  // Lista de tres opciones de respuesta.
	"respuesta_correcta": string  // Índice de la opción correcta (0, 1 o 2).
}
```
        


{'pregunta': '¿Cuál de las siguientes afirmaciones sobre LangChain es correcta?',
 'opciones': '0. LangChain solo se puede utilizar con modelos de aprendizaje profundo de código abierto.\n1. LangChain proporciona una interfaz para interactuar con diversas fuentes de datos y modelos de lenguaje.\n2. LangChain es exclusivo para la creación de chatbots simples.',
 'respuesta_correcta': '1'}

# **3. PydanticOutputParser: Convierte la salida a un modelo de datos Pydantic.**

Si buscas simplicidad y rapidez, StructuredOutputParser es una buena opción pero si necesitas validación de datos robusta y estás utilizando Pydantic en tu proyecto, PydanticOutputParser es la mejor alternativa.

| Característica              | StructuredOutputParser               | PydanticOutputParser               |
|-----------------------------|--------------------------------------|-------------------------------------|
| **Definición de esquema**    | Lista de `ResponseSchema`            | Clase Pydantic (`BaseModel`)        |
| **Validación de tipos**      | No estricta                         | Estricta (usando Pydantic)          |
| **Uso**                     | Sencillo y rápido                   | Más verboso pero organizado         |
| **Integración con Pydantic**| No                                  | Sí                                  |
| **Recomendado para**         | Prototipos rápidos                  | Aplicaciones robustas y complejas   |

Veamos como realizar el ejemplo anterior con PydanticOutputParser

In [10]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

# Definimos el modelo Pydantic para la respuesta
class PreguntaTest(BaseModel):
    pregunta: str = Field(description="Texto de la pregunta generada.")
    opciones: List[str] = Field(description="Lista de tres opciones de respuesta.")
    respuesta_correcta: int = Field(description="Índice de la opción correcta (0, 1 o 2).")

# Crear el PydanticOutputParser
output_parser = PydanticOutputParser(pydantic_object=PreguntaTest)

# Obtener el formato de instrucciones del parser
format_instructions = output_parser.get_format_instructions()
format_instructions

# Crear el prompt
texto_prompt = """
Genera una pregunta de test sobre el tema: {tema}.
El nivel de dificultad debe ser: {nivel}.
La pregunta debe tener tres opciones de respuesta y una respuesta correcta.

{format_instructions}
"""

prompt_template = PromptTemplate(
    template=texto_prompt,
    input_variables=["tema", "nivel"],
    partial_variables={"format_instructions": format_instructions}
)

# Generar el prompt para el LLM
prompt = prompt_template.format(tema="LangChain", nivel="medio")
print("Prompt generado:\n", prompt)

# Instanciamos el modelo
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=1)

# Hacemos la llamada
respuesta = llm.invoke(prompt)

# Analizamos la respuesta
respuesta_analizada = output_parser.parse(respuesta.content)
print("Respuesta analizada:\n", respuesta_analizada)


Prompt generado:
 
Genera una pregunta de test sobre el tema: LangChain.
El nivel de dificultad debe ser: medio.
La pregunta debe tener tres opciones de respuesta y una respuesta correcta.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"pregunta": {"description": "Texto de la pregunta generada.", "title": "Pregunta", "type": "string"}, "opciones": {"description": "Lista de tres opciones de respuesta.", "items": {"type": "string"}, "title": "Opciones", "type": "array"}, "respuesta_correcta": {"description": "Índice de la opción correcta (0, 1 o 2).", "title": "Respuesta Cor

Veamos otro ejemplo con Pydantic. Deseamos obtener la bibliografia ordenada del autor indicado. deseamos una lista ordenada con el año y el titulo de cada libro.





In [17]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

# Definimos el modelo Pydantic para la bibliografía
class Libro(BaseModel):
    año: int = Field(description="Año de publicación del libro.")
    título: str = Field(description="Título del libro.")

class BibliografiaAutor(BaseModel):
    libros: List[Libro] = Field(description="Lista de libros publicados por el autor.")

# Crear el PydanticOutputParser
output_parser = PydanticOutputParser(pydantic_object=BibliografiaAutor)

# Obtener el formato de instrucciones del parser
format_instructions = output_parser.get_format_instructions()
format_instructions

# Crear el prompt
texto_prompt = """
Proporciona la bibliografía del autor: {autor}.
La respuesta debe incluir solo el año y el título de cada libro.
No incluyas información adicional.

{format_instructions}
"""

prompt_template = PromptTemplate(
    template=texto_prompt,
    input_variables=["autor"],
    partial_variables={"format_instructions": format_instructions}
)

# Generar el prompt para el LLM
autor="George R. R. Martin"
prompt = prompt_template.format(autor=autor)
print("Prompt generado:\n", prompt)

# Instanciamos el modelo
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=1)

# Hacemos la llamada
respuesta = llm.invoke(prompt)

display(Markdown(f"**Respuesta del LLM sin parsear:**\n{respuesta.content}"))

# Analizamos la respuesta
respuesta_analizada = output_parser.parse(respuesta.content)

# Imprimir la bibliografía de forma legible
# La función enumerate toma un iterable (como una lista) y devuelve un objeto que genera pares de valores
# start=1 , para que el 0 se considere 1
print(f"Bibliografía de {autor}:\n")
for i, libro in enumerate(respuesta_analizada.libros, start=1):
    print(f"Libro {i}:")
    print(f"  Año: {libro.año}")
    print(f"  Título: {libro.título}\n")

print(type(respuesta_analizada))

Prompt generado:
 
Proporciona la bibliografía del autor: George R. R. Martin.
La respuesta debe incluir solo el año y el título de cada libro.
No incluyas información adicional.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"Libro": {"properties": {"año": {"description": "Año de publicación del libro.", "title": "Año", "type": "integer"}, "título": {"description": "Título del libro.", "title": "Título", "type": "string"}}, "required": ["año", "título"], "title": "Libro", "type": "object"}}, "properties": {"libros": {"description": "Lista de libros publicados por el autor.", "

**Respuesta del LLM sin parsear:**
```json
{"libros":[{"año":1977,"título":"Dying of the Light"},{"año":1981,"título":"Windhaven"},{"año":1983,"título":"Fevre Dream"},{"año":1996,"título":"A Game of Thrones"},{"año":1998,"título":"A Clash of Kings"},{"año":2000,"título":"A Storm of Swords"},{"año":2005,"título":"A Feast for Crows"},{"año":2011,"título":"A Dance with Dragons"},{"año":2020,"título":"Fire & Blood"},{"año":2023,"título":"The Rise of the Dragon"}]}
```

Bibliografía de George R. R. Martin:

Libro 1:
  Año: 1977
  Título: Dying of the Light

Libro 2:
  Año: 1981
  Título: Windhaven

Libro 3:
  Año: 1983
  Título: Fevre Dream

Libro 4:
  Año: 1996
  Título: A Game of Thrones

Libro 5:
  Año: 1998
  Título: A Clash of Kings

Libro 6:
  Año: 2000
  Título: A Storm of Swords

Libro 7:
  Año: 2005
  Título: A Feast for Crows

Libro 8:
  Año: 2011
  Título: A Dance with Dragons

Libro 9:
  Año: 2020
  Título: Fire & Blood

Libro 10:
  Año: 2023
  Título: The Rise of the Dragon

<class '__main__.BibliografiaAutor'>


Observa que la clase del objeto devuelto NO es un diccionario, ni un JSON sino un objeto heredado de la clase que hemos definido con Pydantic !!


# **4. JSONoutputParser**

# **5. CommaSeparatedListOutputParser:**

Analiza la salida del LLM y la convierte en una lista de strings. Puedes especificar un separador para dividir el texto en elementos de la lista.
    
-   **Casos de uso:** Obtener listas de elementos, como nombres, ideas, pasos a seguir, o categorías.
    
-   **Ventaja:** Simple y efectivo para extraer listas.

Vamos a hacer un ejemplo en el que queremos obtener una lista de ingredientes para hacer una pizza casera.



In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser

# Crear el ListOutputParser
output_parser = CommaSeparatedListOutputParser()

# Crear el prompt
prompt = PromptTemplate(
    template="Genera una lista de ingredientes para hacer una pizza casera. Solo lista los ingredientes, uno por línea.\n{format_instructions}\n",
    input_variables=[],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

# Inicializar el modelo de lenguaje
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.7)

# Generar la salida
input_prompt = prompt.format()
respuesta = llm.invoke(input_prompt)

# Parsear la salida
respuesta_parseada = output_parser.parse(respuesta.content)

# Mostrar el resultado
print(respuesta_parseada)

['harina', 'agua', 'levadura', 'sal', 'aceite de oliva', 'salsa de tomate', 'queso mozzarella', 'pepperoni', 'oregano', 'albahaca']


# **6. RegexOutputParser**

# **6. EnumOutputParser**

Es útil cuando esperas que el LLM elija entre un conjunto predefinido de opciones (un enum de Python).

Casos de uso: Clasificación, selección de categorías, o elegir una opción de un menú.

Ventaja: Garantiza que la salida se encuentre dentro de un conjunto de valores válidos. Imaginemos que estamos construyendo un sistema de recomendación de libros, donde el modelo debe elegir un género literario de una lista predefinida y luego recomendar un libro de ese género.

❗ Este ejemplo no funciona ❓ -> porque este parser es mas bien para "forzar" a elegir al modelo una de las opciones. Observa que el "formato" que el parser espera (instructions) se incluye en el prompt para "forzar" la respuesta del modelo y luego el parser espera poder recoger la salida, pero a veces puede no estar bien

In [ ]:
from enum import Enum
from langchain.output_parsers import EnumOutputParser

# Definir las opciones del enumerado
class ColoresOjos(Enum):
    MARRÓN = "marrón"
    AVELANA = "avellana"
    ÁMBAR = "ámbar"
    VERDE = "verde"
    AZUL = "azul"
    GRIS = "gris"


# Crear el EnumOutputParser
output_parser = EnumOutputParser(enum=ColoresOjos)

# Crear el prompt
prompt = PromptTemplate(
    template="De que color tenia o tiene los ojos esta persona: {persona}.",
    input_variables=["persona"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

# Inicializar el modelo de lenguaje
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.7)

# Generar la salida
input_prompt = prompt.format(persona= "Frank Sinatra")
output = llm.invoke(input_prompt).content.lower()

parsed_output = output_parser.parse(output)

# Parsear la salida
print(f"El género elegido es: {parsed_output}")


En este ejemplo realizamos dos llamadas al modelo. En la primera le pedimos que escoja entre una opcion del Enum, y con ella realizamos la segunda invocación.

In [ ]:
from langchain.output_parsers import EnumOutputParser

from enum import Enum

# Definir los géneros literarios como un Enum
class GenerosLiterarios(Enum):
    FANTASÍA = "fantasía"
    CIENCIA_FICCIÓN = "ciencia ficción"
    MISTERIO = "misterio"
    ROMANCE = "romance"
    HISTÓRICA = "novela histórica"
    NO_FICCIÓN = "no ficción"

# Crear el EnumOutputParser
output_parser = EnumOutputParser(enum=GenerosLiterarios)

# Crear el prompt
prompt = PromptTemplate(
    template="Elige un género literario de la siguiente lista: {generos_literarios}.\nResponde solo con el genero escogido \n{format_instructions}\n",
    input_variables=[],
    partial_variables={
        "generos_literarios": ", ".join([g.value for g in GenerosLiterarios]),
        "format_instructions": output_parser.get_format_instructions()
    }
)

# Inicializar el modelo de lenguaje
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=2)

# Generar la salida
input_prompt = prompt.format()
print(input_prompt)
respuesta = llm.invoke(input_prompt).content.lower()
print (respuesta)



Elige un género literario de la siguiente lista: fantasía, ciencia ficción, misterio, romance, novela histórica, no ficción.
Responde solo con el genero escogido 
Select one of the following options: fantasía, ciencia ficción, misterio, romance, novela histórica, no ficción

fantasía


In [ ]:
# Parsear la salida
try:
    genero_elegido = output_parser.parse(respuesta)
    print(f"El género elegido es: {genero_elegido.value}")

    # Ahora pedimos una recomendación de libro para el género elegido
    recomendacion_prompt = PromptTemplate(
        template="Recomienda un libro de género {genero}.",
        input_variables=["genero"]
    )
    recomendacion_input = recomendacion_prompt.format(genero=genero_elegido.value)
    recomendacion = llm.invoke(recomendacion_input)

    print(f"Recomendación: {recomendacion.content}")
except ValueError as e:
    print(f"Error: {e}")

No esta demasiado clara la conveniencia de ests OutputParser. Porque no hacer simplemente una eleccion aleatoria  ¿?¿?¿?


```
genero_elegido=random.choice(list(GenerosLiterarios))
```
En lugar de invocar al modelo ¿?¿?¿?





# **BooleanOutputParser**



In [ ]:
from langchain.output_parsers import BooleanOutputParser


# Crear el BooleanOutputParser
output_parser = BooleanOutputParser()

# Crear el prompt
prompt = PromptTemplate(
    template="Responde con 'Yes' o 'No' a la siguiente afirmación: 'El sol es una estrella.'\n{format_instructions}\n",
    input_variables=[],
    partial_variables={"format_instructions":" "}
)

# Inicializar el modelo de lenguaje
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.7)

# Generar la salida
input_prompt = prompt.format()
respuesta = llm.invoke(input_prompt)

# Parsear la salida
respuesta_parseada = output_parser.parse(respuesta.content)

# Mostrar el resultado
print(respuesta_parseada)

True


In [ ]:
from langchain.output_parsers import RegexParser


# Definir la expresión regular para extraer nombre y edad
regex_pattern = r"Nombre: (?P<nombre>.+)\nEdad: (?P<edad>\d+)"

# Crear el RegexParser
output_parser = RegexParser(regex=regex_pattern, output_keys=["nombre", "edad"])

# Crear el prompt
prompt = PromptTemplate(
    template="Genera información de una persona ficticia, incluyendo su nombre y edad.\n{format_instructions}\n",
    input_variables=[],
    partial_variables={"format_instructions": ""}
)

# Inicializar el modelo de lenguaje
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.7)

# Generar la salida
input_prompt = prompt.format()
respuesta = llm.invoke(input_prompt)

# Parsear la salida
respuesta_parseada = output_parser.parse(respuesta.content)

# Mostrar el resultado
print(respuesta_parseada)

{'nombre': 'Laura Martínez  ', 'edad': '28'}


# **OutputFixingParser**

Referencias:

1. https://freedium.cfd/https://python.plainenglish.io/langchain-in-chains-7-output-parsers-e1a2cdd40cd3

2. https://bobrupakroy.medium.com/harness-llm-output-parsers-for-a-structured-ai-7b456d231834

3. https://cobusgreyling.medium.com/langchain-structured-output-parser-using-openai-c3fe6927beb7

4. https://python.langchain.com/docs/how_to/output_parser_structured/

5. https://www.comet.com/site/blog/mastering-output-parsing-in-langchain/

6. https://www.gettingstarted.ai/how-to-langchain-output-parsers-convert-text-to-objects/

7. https://www.gettingstarted.ai/how-to-extract-metadata-from-pdf-convert-to-json-langchain/  Este es un buen reto

8. https://www.analyticsvidhya.com/blog/2024/11/output-parsers/
